In [1]:
import dotenv
from transformers import LongformerTokenizer, LongformerModel
import torch
from langchain_pinecone import PineconeVectorStore
from langchain_anthropic import ChatAnthropic
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.utilities import PythonREPL

# Load environment variables
dotenv.load_dotenv()

/Users/rahulsingh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/rahulsingh/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
# Load Longformer model and tokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-large-4096')
model = LongformerModel.from_pretrained('allenai/longformer-large-4096')

In [3]:
# Define custom embedding class with embed_query method
class LongformerEmbedding:
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model

    def embed_query(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", max_length=4096, truncation=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()  # Mean pooling
        return embedding

# Instantiate the custom embedding class
longformer_embedder = LongformerEmbedding(tokenizer, model)

In [4]:
# Connect to Pinecone and initialize with the existing index
index_name = "hubspot-crm-txts"

# Initialize PineconeVectorStore with the custom embedding class
vector_store = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=longformer_embedder  # Pass the embedding class here
)

In [98]:
# Set up the Retrieval Chain with LangChain
llm_rag = ChatAnthropic(model='claude-3-5-sonnet-20241022', temperature=0.1)

# Define your custom prompt with context and prefix
rag_template = """
    Role: HubSpot API Python Code Generator.

    Context:
    {text}

    Generate Python code that:
    - Uses the HubSpot Client Library when possible; falls back to `requests` for unsupported features.
    - Returns raw JSON responses only.
    - Uses the provided `access_token` and includes necessary imports.
    - Ends with `print(response)`.
    - Wraps all operations in `try-except` for error handling.
    - Avoids comments, docstrings, and markdown.

    Special Considerations:
    - Filtering:
    - Use Client Library filters with correct imports if supported.
    - Otherwise, send filter payloads via `requests`.
    - Associations:
    - Use `requests.get()` for endpoints formatted as:
        https://api.hubapi.com/crm/v4/objects/<object_type>/<id>/associations/<to_object_type>.
    - Include an `Authorization` header with the Bearer token.

    Question:
    {question}
"""

rag_prompt = PromptTemplate(template=rag_template, input_variables=["text", "question"])

In [99]:
# Define the Retrieval-Augmented Generation (RAG) chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm_rag,
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
    chain_type="stuff",
    chain_type_kwargs = {
        "prompt": rag_prompt,
        "document_variable_name": "text"
    },   
)

In [100]:
# Setup simple llm to explain the results
llm_explanation = ChatAnthropic(model='claude-3-5-haiku-20241022', temperature=0.5)

explanation_template = """
    Explain the HubSpot response below in plain language to answer the user's query:
    
    Response: {response}
    Query: {query}

    Format:
    1. Summary line: "Found [X] results for [query type]"
    2. Bullet-point details for each result
    3. For errors (if any): Explain issue and solution simply

    Rules:
    - Use everyday language
    - Be thorough but simple
    - No clarification offers
    - Instead of saying API request, say "ask me about" or "make a request for"

    Style:
    - Clear
    - Descriptive
    - Easy to understand
    - Direct
"""

explanation_prompt = PromptTemplate(template=explanation_template, input_variables=["response", "query"])

In [101]:
# Define the explanation chain
explanation_chain = LLMChain(
    llm=llm_explanation,
    prompt=explanation_prompt
)

In [102]:
# Query the RAG Chain
query = "tell me about the contacts assoicated with the deal 'HubSpot - New Deal (Sample Deal)'. my access token is pat-na1-59c263ed-698c-4077-b2bc-d1e001750420"
response = rag_chain.invoke(query)

In [103]:
# Run the explanation chain
explanation = explanation_chain.run({
    "response": PythonREPL().run(response['result']),
    "query": query
})
print(explanation)

Summary: Found 1 contact associated with the deal

Details:
• The contact linked to this deal has the ID 71750622225
• The association is a standard HubSpot-defined connection (type ID 3)

Note: This response only shows the contact ID. To get more specific details about the contact (like name, email), you would need to make another request to retrieve the full contact information using this ID.

No errors detected in the response.


In [104]:
print(response['result'])

```python
from hubspot import HubSpot
import requests

access_token = 'pat-na1-59c263ed-698c-4077-b2bc-d1e001750420'

try:
    hubspot = HubSpot(access_token=access_token)
    
    # First get the deal ID by searching for the deal name
    filter_dict = {
        "filterGroups": [{
            "filters": [{
                "propertyName": "dealname",
                "operator": "EQ",
                "value": "HubSpot - New Deal (Sample Deal)"
            }]
        }]
    }
    
    deals_response = hubspot.crm.deals.search_api.do_search(
        filter_dict
    )
    
    if deals_response.results:
        deal_id = deals_response.results[0].id
        
        # Get associated contacts using requests
        url = f"https://api.hubapi.com/crm/v4/objects/deals/{deal_id}/associations/contacts"
        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json"
        }
        
        response = requests.get(url, headers=headers).